In [1]:
from pyvis.network import Network
import networkx as nx
import json
import functools
import itertools
import collections
from matplotlib import pyplot as plt
from networkx.drawing.nx_agraph import write_dot, graphviz_layout

In [2]:
# utility functions
def none_max(a, b):
    if a is None:
        return b
    if b is None:
        return a
    return max(a, b)

def max_dict(dict_a, dict_b):
   all_keys = dict_a.keys() | dict_b.keys()
   return  {k: none_max(dict_a.get(k), dict_b.get(k)) for k in all_keys}

In [3]:
def plot_graph(G,  heading, layout=False, width=1000, height=550, physics=True):

    g = Network(height=height,
                width=width,
                notebook=True,
                directed=True,
                heading=heading,
                layout=layout)

    # level_dict = nx.nx.single_source_shortest_path_length(G.reverse(), num_mapping["fin:end"])
    # [g.add_node(k, level=5-v) for k, v in level_dict.items()]
    # level_dict[num_mapping["fin:end"]] = len(nx.dag_longest_path(G))-1
    # [g.add_node(k, level=v+1) for k, v in level_dict.items()]
    g.toggle_physics(physics)
    nodes = list(G.nodes())
    root_nodes = [n for n,d in G.in_degree() if d==0]
    level_dict = {i: 0 for i in root_nodes}
    level_dict_two = (
        functools.reduce(lambda a,b : max_dict(a,b),
                                      [{j: max([len(k) for k in nx.all_simple_paths(G, i, j)])
                                        for j in nx.descendants(G, i)}
                                       for i in root_nodes]
        )
    )
    level_dict.update(level_dict_two)
    [g.add_node(k, level=v) for k, v in level_dict.items()]

    g.from_nx(G)
    return g

In [4]:
G = nx.DiGraph()

In [5]:
G.add_edges_from([(1,2), (1,3), (2,4), (3,4), (1,4)])

In [6]:
plot_graph(G, heading="DAG", layout=True).show("./test.html")

In [7]:
dict(G.in_degree)

{1: 0, 2: 1, 3: 1, 4: 3}

In [8]:
TR = nx.DiGraph()

In [9]:
TR.add_nodes_from(G.nodes())

In [10]:
plot_graph(TR, heading="TR", layout=True).show("ex.html")

In [11]:
for u in G:
    print(set(G[u]))

{2, 3, 4}
{4}
{4}
set()


In [12]:
import json


def print_vars(d):
    for i in d:
        print("%-20s %s" % (str(i), str(d[i])))
    print()


def transitive_reduction(G):
    """ Returns transitive reduction of a directed graph

    The transitive reduction of G = (V,E) is a graph G- = (V,E-) such that
    for all v,w in V there is an edge (v,w) in E- if and only if (v,w) is
    in E and there is no path from v to w in G with length greater than 1.

    Parameters
    ----------
    G : NetworkX DiGraph
        A directed acyclic graph (DAG)

    Returns
    -------
    NetworkX DiGraph
        The transitive reduction of `G`

    Raises
    ------
    NetworkXError
        If `G` is not a directed acyclic graph (DAG) transitive reduction is
        not uniquely defined and a :exc:`NetworkXError` exception is raised.

    References
    ----------
    https://en.wikipedia.org/wiki/Transitive_reduction

    """
    TR = nx.DiGraph()
    TR.add_nodes_from(G.nodes())
    descendants = {}
    check_count = dict(G.in_degree)
    print_vars({'check_count': check_count})
    for u in G:
        u_nbrs = set(G[u])
        print_vars({'u': str(u), 'u_nbrs': u_nbrs})
        for v in G[u]:
            if v in u_nbrs:
                if v not in descendants:
                    descendants[v] = {y for x,y in nx.dfs_edges(G, v)}
                    print_vars({'v': str(v), 'descendants': descendants})
                u_nbrs -= descendants[v]
            check_count[v] -= 1
            print_vars({'v': v, 'u_nbrs': u_nbrs, 'check_count': check_count})
            if check_count[v] == 0:
                del descendants[v]
        TR.add_edges_from((u, v) for v in u_nbrs)
        print_vars({'TR Edges': list(TR.edges())})
    return TR

In [13]:
G = nx.DiGraph()
G.add_edges_from([(1,2), (1,3), (2,4), (3,4), (1,4)])
TR = transitive_reduction(G)

check_count          {1: 0, 2: 1, 3: 1, 4: 3}

u                    1
u_nbrs               {2, 3, 4}

v                    2
descendants          {2: {4}}

v                    2
u_nbrs               {2, 3}
check_count          {1: 0, 2: 0, 3: 1, 4: 3}

v                    3
descendants          {3: {4}}

v                    3
u_nbrs               {2, 3}
check_count          {1: 0, 2: 0, 3: 0, 4: 3}

v                    4
u_nbrs               {2, 3}
check_count          {1: 0, 2: 0, 3: 0, 4: 2}

TR Edges             [(1, 2), (1, 3)]

u                    2
u_nbrs               {4}

v                    4
descendants          {4: set()}

v                    4
u_nbrs               {4}
check_count          {1: 0, 2: 0, 3: 0, 4: 1}

TR Edges             [(1, 2), (1, 3), (2, 4)]

u                    3
u_nbrs               {4}

v                    4
u_nbrs               {4}
check_count          {1: 0, 2: 0, 3: 0, 4: 0}

TR Edges             [(1, 2), (1, 3), (2, 4), (3, 4)]

u          

In [14]:
plot_graph(TR, layout=True, heading="TR").show("tr.html")

The algorithm in plain english

for every node u  
visit all children of u, let the children be v  
maintain a set u_nbrs, which has all children of v  
Use nx.dfs_edges to get all descendents of v  
Delete all descendents of v from u_nbrs  
As there is a longer path from u to descendants of v through v  
After performing the above operation for all children of u  
Add the edges from u to every node in u_nbrs  